In [ ]:
# First reset the pygkyl library
import sys
!rm -rf ~/personal_gkyl_scripts/pygkyl/pygkyl.egg-info
!rm -rf ~/personal_gkyl_scripts/pygkyl/build
!{sys.executable} -m pip install ~/personal_gkyl_scripts/pygkyl > ~/personal_gkyl_scripts/pygkyl/install.log

import numpy as np
import matplotlib.pyplot as plt
import os

# Configure plotting
plt.rcParams["figure.figsize"] = (5,4)

# Custom libraries and routines
import pygkyl

home_dir = os.path.expanduser("~")
repo_dir = home_dir+'/personal_gkyl_scripts/'
simdir = '/Users/ahoffman/personal_gkyl_scripts/sim_data/3x2v/gk_tcv_adapt_src/'
fileprefix = 'rt_gk_tcv_nt_iwl_3x2v_p1'
simulation = pygkyl.simulation_configs.import_config('tcv_nt', simdir, fileprefix)
simulation.normalization.set('t','mus') # time in micro-seconds
simulation.normalization.set('x','minor radius') # radial coordinate normalized by the minor radius (rho=r/a)
simulation.normalization.set('y','Larmor radius') # binormal in term of reference sound Larmor radius
simulation.normalization.set('z','pi') # parallel angle devided by pi
simulation.normalization.set('fluid velocities','thermal velocity') # fluid velocity moments are normalized by the thermal velocity
simulation.normalization.set('temperatures','eV') # temperatures in electron Volt
simulation.normalization.set('pressures','Pa') # pressures in Pascal
simulation.normalization.set('energies','MJ') # energies in mega Joules
simulation.normalization.set('gradients','major radius') # gradients are normalized by the major radius
# simulation.normalization.set('vpar','vt')
# simulation.normalization.set('mu','mu0')

frames_conf = simulation.available_frames['field'] # you can check the available frames for each data type like ion_M0, ion_BiMaxwellian, etc.)
frames_phsp = simulation.available_frames['ion'] # you can check the available frames for each data type like ion_M0, ion_BiMaxwellian, etc.)
print("Config frames: ", frames_conf)
print("Phase-space frames: ", frames_phsp)

In [ ]:
def load_frames(simulation, timeframe):
    Bmag = pygkyl.Frame(simulation=simulation,fieldname='Bmag',tf=timeframe,load=True)
    phi = pygkyl.Frame(simulation=simulation,fieldname='phi',tf=timeframe,load=True)
    ne = pygkyl.Frame(simulation=simulation,fieldname='ne',tf=timeframe,load=True)
    ni = pygkyl.Frame(simulation=simulation,fieldname='ni',tf=timeframe,load=True)
    Te = pygkyl.Frame(simulation=simulation,fieldname='Te',tf=timeframe,load=True)
    Ti = pygkyl.Frame(simulation=simulation,fieldname='Ti',tf=timeframe,load=True)
    fe = pygkyl.Frame(simulation=simulation,fieldname='fe',tf=timeframe,load=True)
    fi = pygkyl.Frame(simulation=simulation,fieldname='fi',tf=timeframe,load=True)
    gamma = pygkyl.Frame(simulation=simulation,fieldname='rhoe_lambdaD',tf=timeframe,load=True)
    return Bmag, phi, ne, ni, Te, Ti, fe, fi, gamma

def eval_frames(frames, x, y, z):
    values = []
    for frame in frames:
        if frame.dimensionality == 3:
            values.append(frame.get_values([x, y, z]))
        elif frame.dimensionality == 5:
            values.append(frame.get_values([x, y, z, 'all', 'all']))
            
def get_grids(distf_frame):
    xgrid = distf_frame.new_grids[0]
    ygrid = distf_frame.new_grids[1]
    zgrid = distf_frame.new_grids[2]
    vpargrid = distf_frame.new_grids[3]
    mugrid = distf_frame.new_grids[4]
    return xgrid, ygrid, zgrid, vpargrid, mugrid

def get_ranges(xgrid, ygrid, zgrid, xmin, xmax, Nx, Ny, zplane):
    ixmin = np.argmin(np.abs(xgrid - xmin))
    ixmax = np.argmin(np.abs(xgrid - xmax))
    iymin = 0
    iymax = len(ygrid)-1
    if zplane=='upper':
        izplane = np.argmax(zgrid)
    elif zplane=='lower':
        izplane = np.argmin(zgrid)
    
    xindices = np.linspace(ixmin, ixmax, Nx, dtype=int)
    yindices = np.linspace(iymin, iymax, Ny, dtype=int)
    # remove duplicates
    xindices = np.unique(xindices)
    yindices = np.unique(yindices)
    return xindices, yindices, izplane

In [ ]:
import h5py

tf = frames_phsp[-1]

xmin = 1.1
xmax = 1.3
Nxsample = 3 # number of x samples
Nysample = 4 # number of y samples
zplane = 'upper' # 'upper' or 'lower' side of the limiter
alphadeg = 0.1 # angle of the magnetic field to the wall in degree
nspec = 2

# ----- END OF USER INPUTS -----

# Load the frames
Bmag, phi, ne, ni, Te, Ti, fe, fi, gamma = load_frames(simulation, tf)
xgrid, ygrid, zgrid, vparegrid, muegrid = get_grids(fe)
xgrid, ygrid, zgrid, vparigrid, muigrid = get_grids(fi)

me = simulation.species['elc'].m
mi = simulation.species['ion'].m
mioverme = mi/me
e = np.abs(simulation.species['elc'].q)

xindices, yindices, izplane = get_ranges(xgrid, ygrid, zgrid, xmin, xmax, Nxsample, Nysample, zplane)
# Sample points in the (x,y) plane
for ix in xindices:
    for iy in yindices:
        print(f'ix={ix}, x0={xgrid[ix]:.3f}, iy={iy}, y0={ygrid[iy]:.3f}, iz={izplane}, z0={zgrid[izplane]:.3f}')
        x0 = xgrid[ix]
        y0 = ygrid[iy]
        z0 = zgrid[izplane]
        
        B0, phi0, ne0, ni0, Te0, Ti0, fe0, fi0, gamma0 = eval_frames([Bmag, phi, ne, ni, Te, Ti, fe, fi, gamma], x0, y0, z0)

        nioverne = ni0/ne0
        TioverTe = Ti0/Te0

        vpnorme = vparegrid / np.sqrt(Te0*e/me)
        munorme = muegrid / (Te0*e/B0)
        vpnormi = vparigrid / np.sqrt(Ti0*e/mi)
        munormi = muigrid / (Ti0*e/B0)

        ## Filter negativity in fe0 and fi0
        fe0[fe0<0] = 0.0
        fi0[fi0<0] = 0.0

        # Create Fe_mpe_args.txt file
        with open('Fe_mpe_args.txt', 'w') as f:
            # Write xperp values in the first line
            f.write(' '.join(map(str, munorme)) + '\n')
            # Write spar values in the second line
            f.write(' '.join(map(str, vpnorme)))
        # Write Fe_mpe.txt file with electron distribution function values
        np.savetxt('Fe_mpe.txt', fe0.squeeze().T)
                    
        # Same for the ions
        with open('Fi_mpi_args.txt', 'w') as f:
            # Write xperp values in the first line
            f.write(' '.join(map(str, munormi)) + '\n')
            # Write spar values in the second line
            f.write(' '.join(map(str, vpnormi)))  
        np.savetxt('Fi_mpi.txt', fi0.squeeze().T)

        # Create input_physparams.txt file with the physical parameters
        with open('input_physparams.txt', 'w') as f:
            f.write('#set type_distfunc_entrance (= ADHOC or other string)\n')
            f.write('GKEYLL\n')
            f.write('#set alphadeg\n')
            f.write(f'{alphadeg}\n')
            f.write('#set gamma_ref (keep zero to solve only magnetic presheath)\n')
            f.write(f'{gamma0}\n')
            f.write('#set nspec\n')
            f.write(f'{nspec}\n')
            f.write('#set nioverne\n')
            f.write(f'{nioverne}\n')
            f.write('#set TioverTe\n')
            f.write(f'{TioverTe}\n')
            f.write('#set mioverme\n')
            f.write(f'{mioverme}\n')
            f.write('#set set_current (flag)\n')
            f.write('0\n')
            f.write('#set target_current or phi_wall\n')
            f.write(f'{phi0}\n')
            
        # Read the contents of the text files
        with open('Fe_mpe_args.txt', 'r') as f:
            fe_mpe_args_text = f.read()
        with open('Fe_mpe.txt', 'r') as f:
            fe_mpe_text = f.read()
        with open('Fi_mpi_args.txt', 'r') as f:
            fi_mpi_args_text = f.read()
        with open('Fi_mpi.txt', 'r') as f:
            fi_mpi_text = f.read()
        with open('input_physparams.txt', 'r') as f:
            input_physparams_text = f.read()

        # Create an HDF5 file and store the data from the text files
        with h5py.File('data.h5', 'w') as hf:
            # Store text file contents as strings
            hf.create_dataset('Fe_mpe_args.txt', data=fe_mpe_args_text, dtype=h5py.string_dtype(encoding='utf-8'))
            hf.create_dataset('Fe_mpe.txt', data=fe_mpe_text, dtype=h5py.string_dtype(encoding='utf-8'))
            hf.create_dataset('Fi_mpi_args.txt', data=fi_mpi_args_text, dtype=h5py.string_dtype(encoding='utf-8'))
            hf.create_dataset('Fi_mpi.txt', data=fi_mpi_text, dtype=h5py.string_dtype(encoding='utf-8'))
            hf.create_dataset('input_physparams.txt', data=input_physparams_text, dtype=h5py.string_dtype(encoding='utf-8'))
            
        # Remove the intermediate text files
        os.remove('Fe_mpe_args.txt')
        os.remove('Fe_mpe.txt')
        os.remove('Fi_mpi_args.txt')
        os.remove('Fi_mpi.txt')
        os.remove('input_physparams.txt')

In [ ]:
# print(" spare is between %.2e and %.2e"%(spare.min(),spare.max()))
# print(" xperpe is between %.2e and %.2e"%(xperpe.min(),xperpe.max()))
# print(" spari is between %.2e and %.2e"%(spari.min(),spari.max()))
# print(" xperpi is between %.2e and %.2e"%(xperpi.min(),xperpi.max()))
# print("Temperature at (x,y,z) = (%.2f, %.2f, %.2f) is Te = %.2f eV"%(x0,y0,z0,Te0))
# print("Magnetic field at (x,y,z) = (%.2f, %.2f, %.2f) is B = %.2f T"%(x0,y0,z0,B0))
# print("Electrostatic potential is phi = %.2f V"%(phi0))
# print("Electron thermal speed is vte = %.2e m/s"%(vte0))
# print("Reference elc magnetic moment is mu0 = %.2f eV/T"%(mu0e/e))
# print("Ion thermal speed is vti = %.2e m/s"%(vti0))
# print("Reference ion magnetic moment is mu0i = %.2f eV/T"%(mu0i/e))
# print("Ratio of sound Larmor radius to Debye length is rhoe/lambdaD = %.3f"%(gamma0))
# # check shapes
# print("fe0 shape: ", fe0.shape)
# print("fi0 shape: ", fi0.shape)
# print("spare shape: ", spare.shape)
# print("xperpe shape: ", xperpe.shape)
# print("spari shape: ", spari.shape)
# print("xperpi shape: ", xperpi.shape)

In [ ]:
# # Test that the files are good by loading them and plotting
# # Load the xperp and spar values from Fe_mpe_args.txt
# with open('Fe_mpe_args.txt', 'r') as f:
#     lines = f.readlines()
#     xperp_loaded = np.array([float(x) for x in lines[0].strip().split()])
#     spar_loaded = np.array([float(x) for x in lines[1].strip().split()])

# # Load the distribution function values from Fe_mpe.txt
# fe_loaded = []
# with open('Fe_mpe.txt', 'r') as f:
#     for line in f:
#         row = [float(x) for x in line.strip().split()]
#         fe_loaded.append(row)
# fe_loaded = np.array(fe_loaded)
# fe_loaded = fe_loaded.T  # Transpose to match original orientation

# # Verify the data matches
# print("xperp arrays match:", np.allclose(xperp, xperp_loaded))
# print("spar arrays match:", np.allclose(spar, spar_loaded))
# print("fe arrays match:", np.allclose(fe0[:,:].squeeze(), fe_loaded))

# # Plot the loaded data to verify it looks correct
# plt.figure(figsize=(10, 6))
# plt.subplot(1, 2, 1)
# plt.contourf(spar_loaded, xperp_loaded, fe_loaded.T, levels=50)
# plt.colorbar(label='fe')
# plt.xlabel('spar')
# plt.ylabel('xperp')
# plt.title('Loaded Distribution Function')

# plt.subplot(1, 2, 2)
# plt.contourf(spar, xperp, fe0[:,:].squeeze().T, levels=50)
# plt.colorbar(label='fe')
# plt.xlabel('spar')
# plt.ylabel('xperp')
# plt.title('Original Distribution Function')

# plt.tight_layout()
# plt.show()

In [ ]:
# pygkyl.plot_utils.plot_2D_cut(simulation,
#                               cut_dir='xy',
#                               cut_coord=z0,
#                               time_frame=tf,
#                               fieldnames=['Te','phi','Bmag'],
#                               xlim = [1.0,1.3],
#                               clim=[[0,100],[0,300],[1.8,2.1],[0,0]])
# pygkyl.plot_utils.plot_2D_cut(simulation,'vparmu',[x0,y0,z0],tf, ['fe','fi'])